# Resolving Ambiguity in Prepositional Phrase Attachment

The problem of resolving ambiguity in prepositional phrase attachment is one that remains largely unsolved in NLP, and one that pre-trained language models such as BERT will likely not be of much help with. This notebook shows results of predicting prepositional phrase attachments across a subset of the NLVR2 dataset which has been annotated, leveraging a pre-trained language model commonly known as "BERT" (cite). 

The first group of models are trained from the output (hidden layers) of the large uncased model from BERT with whole word masking. The results are presented in terms of Cohen's kappa score and F1 score. 

The second group of models are trained without the aid of a language model. 

Our expectation is that none of these models will perform very well on its own. Results should be comparable between the two groups. 


In [1]:
from IPython.display import Image

## Preliminary Steps

In [2]:
# conda create -n python=3.7 ...
# pip install transformers... 

In [3]:
import sys
import os
import json
import numpy as np

import sklearn
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import cohen_kappa_score as kappa

from itertools import groupby
from collections import Counter

In [4]:
np.random.seed(91768)


In [5]:
datadir = "data"

In [6]:
train_data = json.load(open('{}/ppa_train.json'.format(datadir)))
test_data = json.load(open('{}/ppa_test.json'.format(datadir)))


In [7]:
from generator import HuggingFaceGenerator

In [8]:
labels_train = [instance['label'] for instance in train_data]
labels_test = [instance['label'] for instance in test_data]

In [9]:
bert_model_name = "bert-large-uncased-whole-word-masking"

In [10]:
hf_generator = HuggingFaceGenerator(bert_model_name)

In [12]:
train = hf_generator.generate_dataset(train_data)
test = hf_generator.generate_dataset(test_data)

In [13]:
clfhf = svm.SVC(gamma=0.0001, C=100., random_state=91768)

clfhf.fit(train, labels_train)

SVC(C=100.0, gamma=0.0001, random_state=91768)

In [14]:
preds_test_hf = clfhf.predict(test)
kappa(labels_test, preds_test_hf)

0.6134147542598247

In [15]:
f1_score(labels_test, preds_test_hf, labels=['N','V','O'], average=None)

array([0.90909091, 0.68656716, 0.5       ])